In [4]:
import rasterio

# URL to the COG file on AWS S3
cog_url = 'https://ccemp-bucket.s3.amazonaws.com/Public%20Access/COG/Sudbury-DTM-11_COG_V3.tif'

# Open the COG and print metadata
with rasterio.open(cog_url) as src:
    print("Raster Width:", src.width)
    print("Raster Height:", src.height)
    print("Raster CRS:", src.crs)
    print("Raster Bounds:", src.bounds)  # Geographical bounds of the raster


Raster Width: 26000
Raster Height: 56000
Raster CRS: PROJCS["NAD83 / UTM zone 17N",GEOGCS["NAD83",DATUM["North American Datum 1983",SPHEROID["GRS 1980",6378137,298.257222101004]],PRIMEM["Greenwich",0],UNIT["degree",0.0174532925199433,AUTHORITY["EPSG","9122"]]],PROJECTION["Transverse_Mercator"],PARAMETER["latitude_of_origin",0],PARAMETER["central_meridian",-81],PARAMETER["scale_factor",0.9996],PARAMETER["false_easting",500000],PARAMETER["false_northing",0],UNIT["metre",1,AUTHORITY["EPSG","9001"]],AXIS["Easting",EAST],AXIS["Northing",NORTH]]
Raster Bounds: BoundingBox(left=484000.0, bottom=5165000.0, right=497000.0, top=5193000.0)


## DTM

In [5]:
import os
import rasterio
from rasterio.windows import Window

# URL to the COG file on AWS S3
cog_url = 'https://ccemp-bucket.s3.amazonaws.com/Public%20Access/COG/Sudbury-DTM-11_COG_V3.tif'

# Determine the default download folder
download_folder = os.path.join(os.path.expanduser("~"), "Downloads")

print("Opening COG file...")
with rasterio.open(cog_url) as src:
    bounds = src.bounds
    
    print("Calculating window...")
    center_x, center_y = (bounds.left + bounds.right) / 2, (bounds.bottom + bounds.top) / 2
    block_size = 2000  # In meters
    window = src.window(center_x - block_size/2, center_y - block_size/2, 
                        center_x + block_size/2, center_y + block_size/2)
    
    print("Reading data...")
    data = src.read(1, window=window)

    print("Preparing metadata...")
    meta = src.meta.copy()
    meta.update({
        "height": window.height,
        "width": window.width,
        "transform": src.window_transform(window),
        "compress": "lzw",
        "predictor": 2,
        "tiled": True,
        "blockxsize": 256,
        "blockysize": 256
    })

    output_file = os.path.join(download_folder, "center_4x4_dem_sudbury.tif")
    
    print("Writing output file...")
    with rasterio.open(output_file, 'w', **meta) as dst:
        dst.write(data, 1)

print(f"DEM subset saved as {output_file}")

Opening COG file...
Calculating window...
Reading data...
Preparing metadata...
Writing output file...
DEM subset saved as /Users/b/Downloads/center_4x4_dem_sudbury.tif


## Hillshade (RGB )

In [2]:
import os
import sys
import json
import rasterio
from rasterio.windows import from_bounds
from rasterio.enums import Resampling

# URL to the COG file on AWS S3
cog_url = 'https://ccemp-bucket.s3.amazonaws.com/Public%20Access/COG/Sudbury-DTM-11_COG_V2.tif'

# Determine the default download folder based on the operating system
if sys.platform == "darwin":  # macOS
    download_folder = os.path.join(os.path.expanduser("~"), "Downloads")
elif sys.platform == "win32":  # Windows
    download_folder = os.path.join(os.path.expanduser("~"), "Downloads")
else:
    raise OSError("Unsupported operating system")

# Open the COG file and get its metadata
with rasterio.open(cog_url) as src:
    bounds = src.bounds
    transform = src.transform
    
    # Calculate the center of the raster and define a 5x5 meter block around it
    center_x, center_y = (bounds.left + bounds.right) / 2, (bounds.bottom + bounds.top) / 2
    block_size = 5000  # In meters
    x1, y1, x2, y2 = center_x - block_size / 2, center_y - block_size / 2, center_x + block_size / 2, center_y + block_size / 2


    # Convert the geographic bounding box to a pixel window
    window = from_bounds(x1, y1, x2, y2, transform)
    
    # Adjust the metadata to match the window size and add compression
    meta = src.meta.copy()
    meta.update({
        "driver": "GTiff",
        "height": window.height,
        "width": window.width,
        "transform": src.window_transform(window),
        "count": 3,  # RGB has 3 bands
        "dtype": src.dtypes[0],  # Use the original data type to preserve quality
        "compress": "jpeg"  # Apply LZW compression
    })

    # Output file path (where the subset will be saved)
    output_file = os.path.join(download_folder, "center_5x5_rgb_highest_res.tif")
    
    # Write the window to a new GeoTIFF file
    with rasterio.open(output_file, 'w', **meta) as dst:
        for i in range(1, 4):  # RGB has 3 bands
            dst.write(src.read(i, window=window, resampling=Resampling.nearest), i)

print(f"Subset saved as {output_file}")

Bounding box (5x5 meters) around the center: x1=488000.0, y1=5176500.0, x2=493000.0, y2=5181500.0


IndexError: band index 2 out of range (not in (1,))